## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-13-08-48-51 +0000,nyt,Trump Heads to Israel and Then Egypt as Hostag...,https://www.nytimes.com/live/2025/10/12/us/tru...
1,2025-10-13-08-48-43 +0000,wapo,Here are the key developments,https://www.washingtonpost.com
2,2025-10-13-08-48-43 +0000,wapo,Live updates: Hamas releases remaining living ...,https://www.washingtonpost.com/world/2025/10/1...
3,2025-10-13-08-47-01 +0000,nyt,Here’s the latest.,https://www.nytimes.com/live/2025/10/12/world/...
4,2025-10-13-08-42-13 +0000,bbc,What we know about the Gaza ceasefire agreemen...,https://www.bbc.com/news/articles/cvgqx7ygq41o...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2299/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,31
48,china,18
14,hamas,14
2,israel,13
18,hostages,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
180,2025-10-12-12-36-07 +0000,nypost,Hamas ready to begin freeing 20 living Israeli...,https://nypost.com/2025/10/12/world-news/hamas...,98
100,2025-10-12-22-52-05 +0000,nypost,Hillary Clinton heaps rare praise on President...,https://nypost.com/2025/10/12/us-news/hillary-...,90
78,2025-10-13-00-41-23 +0000,bbc,Trump says 'war is over' in Gaza as he flies t...,https://www.bbc.com/news/articles/cn409y125v3o...,89
41,2025-10-13-05-31-15 +0000,nypost,‘War is over’ Trump declares — as he arrives i...,https://nypost.com/2025/10/13/world-news/war-i...,78
2,2025-10-13-08-48-43 +0000,wapo,Live updates: Hamas releases remaining living ...,https://www.washingtonpost.com/world/2025/10/1...,76


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
180,98,2025-10-12-12-36-07 +0000,nypost,Hamas ready to begin freeing 20 living Israeli...,https://nypost.com/2025/10/12/world-news/hamas...
41,47,2025-10-13-05-31-15 +0000,nypost,‘War is over’ Trump declares — as he arrives i...,https://nypost.com/2025/10/13/world-news/war-i...
8,44,2025-10-13-08-27-01 +0000,nyt,Asia Markets Fall After Trump Threatens New Ta...,https://www.nytimes.com/2025/10/12/business/tr...
29,34,2025-10-13-07-05-15 +0000,nypost,Fighting breaks out between Hamas and armed cl...,https://nypost.com/2025/10/13/world-news/fight...
36,30,2025-10-13-06-21-00 +0000,bbc,"Boy, 11, shoots farm worker dead in tragic Sou...",https://www.bbc.com/news/articles/ce86dvr03zqo...
74,26,2025-10-13-01-03-00 +0000,wsj,Answers to frequently asked questions about Th...,https://www.wsj.com/economy/economic-forecasti...
120,26,2025-10-12-20-29-41 +0000,nypost,Washington Post roasts Mamdani’s free NYC bus ...,https://nypost.com/2025/10/12/us-news/washingt...
186,25,2025-10-12-12-00-00 +0000,cbc,Canada-U.S. trade talks moving ‘in the right d...,https://www.cbc.ca/news/politics/canada-tariff...
181,24,2025-10-12-12-33-11 +0000,cbc,Afghanistan says it killed 58 Pakistani soldie...,https://www.cbc.ca/news/world/afghanistan-kill...
112,23,2025-10-12-21-42-29 +0000,nypost,Zelensky suggests Tomahawk missiles could make...,https://nypost.com/2025/10/12/us-news/zelensky...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
